# Ambient Noise Signal Processing Detector
__Contributors:__ Bilguun Bulgan, Coraline Sun, Caitlyn Liu, Isaac Castillo
<br>__Instructors:__ Ryan Kastner, Perry Naughton, Tahiya Salam
<br>__Description:__ This project is a part of the UC San Diego Early Scholars Research Program (ERSP). This detector was built particularly to detect the signals of the snapping shrimp of the La Jolla shore waters. The detector takes in a sound file, calcuates the frequencies with the Fourier Transformation, then returns "a snap" as a unit. The snap should have comprehensive attributes of a sound wave, including the direct arrival of the snap, and its corresponding reflected waves. 

In [151]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal, arange
from scipy.signal import hilbert, find_peaks_cwt
import matplotlib.pyplot as plt
import matplotlib.axes as matax
import matplotlib.mlab as mlab
import sys
from math import exp
from math import log
import operator


In [152]:
#samplingFrequency: 
#read() returns sample rate of wave file as sampFreq (int) and data read from wav file (numpy array)
sampFreq, sound = read('wav5_7.wav');


In [153]:
#taking 5 seconds of sample from 15s - 20sec in file 
#interval have 100 frames towards left and 400 frames towards right
initTime = 0;    # the start time
duration = int(len(sound)/sampFreq);     # duration of time period start at initTime

###DO NOT CHANGE THE NEXT TWO VARIABLES!!! 
#IF CHANGED PLEASE RESET leftOffset TO 50 AND rightOffset to 200
leftOffset = 50;  # how many frames we should expand the interval to the left (for the classfication algorithm)
rightOffset = 200; # how many frames we should expand ther interval to the right (for the classificaiton algorithm)
###DO NOT CHANGE THE LAST TWO VARIABLES!!!

leftOffsetPeak = 100;
rightOffsetPeak = 6000;
windowSize = 0.01; # the size of the window we will ignore after capture the frame above threshold

deadzone = 0 

#start and end will hold the indeces in the sound sample array
start = sampFreq * initTime;
end = start + sampFreq * duration;

#s1 holds the spliced soundsample array
s1 = sound[start:end];

In [154]:
#preparing timeArray for plotting. The plot will have x-axis in seconds (and not in indeces)
timeArray = np.arange(0, end-start, 1)
timeArray = timeArray / sampFreq

In [155]:
#setting up the amplitude threshold
ampThreshold = np.std(s1) * 10

#the boolean array about whether at each indexthe amplitude is above threshold or not
aboveThres = s1 > ampThreshold 

## This displayes the amplitude graph (with the points of interested noted in dots) and the spectrogram


#### Since 

In [156]:
#pick out the time when the amplitude is above threshold
timeArray[aboveThres]
snaps = [];
snapIndex = [];

# current = the first time frame has amplitude above threshold
# snaps = the time where each loud signal above threshold
# snapIndex = the time frame indices of each loud signal above threshold
current = timeArray[aboveThres][0] 
snaps.append(current)
snapIndex.append(current * sampFreq)

# set a 0.01sec time window that slides across the current array
# so that we only collect one time data for each loud signal
for time in timeArray[aboveThres]: 
    
    if (time > current + windowSize ):
        snapIndex.append(time * sampFreq)
        snaps.append(time)
        current = time
        

In [157]:
def getFrequencies(signal):
    length = len(signal) # length of the signal
    #k = arange(length)
    #T = length/sampFreq
    #frq = k/T # two sides frequency range
    #frq = frq[range(np.int(length/2))] # one side frequency range

    frequencies = np.fft.fft(signal)/length # fft computing and normalization
    frequencies = frequencies[range(np.int(length/2))]
    
    return abs(frequencies)

In [158]:
def plotFrequencies(signal, ax):
    length = len(signal) # length of the signal
    k = arange(length)
    T = length/sampFreq
    frq = k/T # two sides frequency range
    frq = frq[range(np.int(length/2))] # one side frequency range
 
    
    ax.plot(frq, getFrequencies(signal),'r') # plotting the spectrum

    ax.set_xlabel('Freq (Hz)')
    ax.set_ylabel('|Y(freq)|')

In [159]:
def plotAmplitude(signal, time, ax):
    
    ax.plot(time, signal)
    ax.set_xlabel('sample frames')
    ax.set_ylabel('Amplitude')
    ax.set_xlim(front, back)
    ax.margins(x=0)

In [160]:
def plotEnvelope(signal, time, ax):
    analytic_signal = hilbert(signal)
    amplitude_envelope = np.abs(analytic_signal)
   
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * sampFreq)

    ax.plot(time, signal, label='signal')
    ax.plot(time, amplitude_envelope, label='envelope')
    ax.set_xlabel("time in seconds")
    ax.legend()
    
    return markPeaks(amplitude_envelope, time, ax)

In [161]:
def markPeaks(enveloped_signal, time, ax):
    #peak_indexes contain the indeces of all the peaks found in the enveloped signal snippet 
    peak_indexes = find_peaks_cwt(enveloped_signal, arange(1, 20), noise_perc = 50);
    
    #tuple_array will contain a list of tuples in the form of 
    #(<index of the peak>, <amplitude value of the enveloped signal peak>)
    tuple_array=[]
    
    
    #print("x")
    #print(peak_indexes)
    #print("y") 
    #print(enveloped_signal[peak_indexes])
 
    #populates the tuple_array
    for i in peak_indexes:
        data = (time[i],enveloped_signal[i])
        tuple_array.append(data)
    
    #sorts the tuple_array by the 2nd element (which is the amplitude)
    #so the leftmost element would be the peak with the highest
    #amplitude value
    tuple_array.sort(key = operator.itemgetter(1), reverse=True)

    #selected would contain the highest amplitude peak (at most 3, at least 2)
    selected = []
    #ampPairs = []
    
    count = 0
    index = tuple_array[0][0]
    selected.append(tuple_array[0])
    for tuple_element in tuple_array:
        if (tuple_element[0] > index + deadzone/sampFreq):
            #ampPairs.append()
            selected.append(tuple_element)
            index = tuple_element[0]
            count = count + 1
        if (count == 2):
            break;

    index_list = [x[0] for x in selected]

    amplitude_list = [x[1] for x in selected]
    ax.scatter(index_list, amplitude_list, c ='r')
    #print(time[0], time[len(time)-1])
    #print(index_list)
    ax.set_xlim(time[0], time[len(time)-1])

    
    return selected


In [162]:
 def filterNLargestPeaks(n, peak_indexes, enveloped_signal):
    highestPeaks = enveloped_signal[peak_indexes]
    return peak_indexes[np.argsort(highestPeaks)[-n:]]

In [163]:
def getRatio(signal):
    frequencies = getFrequencies(signal)
    ratio = sum(frequencies[34:68] * 100/sum(frequencies))
    return ratio;
    

In [164]:
def classify(signal):
    #print(len(signal))
    
    ratio = getRatio(signal)
    #print(ratio)
    if (ratio > 70):
        return "Ping"
    
    elif (ratio < 50):
        return "Snap"
    else:
        return "Undefined"

In [167]:
timeInterval = []  # the time array for each sound clip
signalInterval = []
freq = []
tuple1 = []
tuple2 = []


signal_count = 0

for i in range(0, len(snapIndex)):
#for i in range(0, 50):
    front= int(snapIndex[i]-leftOffset)
    back=  int(snapIndex[i]+rightOffset)
    
    frontPeak = int(snapIndex[i]-leftOffsetPeak)
    backPeak = int(snapIndex[i]+rightOffsetPeak)
    
    time = timeArray[frontPeak:backPeak];
    
    signalForClassification = s1[front:back]
    classification = classify(signalForClassification)
    
    fig, axes = plt.subplots(nrows=2, ncols=1)
    
    #--- Figure 1. amplitude - time ---#
    #plotAmplitude(signal, time, axes[0])
    
    signalForPeaks = s1[frontPeak:backPeak]
    
    signal_pairs = plotEnvelope(signalForPeaks, time, axes[0])

    
    #--- Figure 2. Y(freq) - freq ---#
    plotFrequencies(signalForPeaks, axes[1])
    
    
    #--- Scatter Plot ---#
    if (classification == "Snap"):
        amp = s1[int((signal_pairs[0][0])*sampFreq)]
        for i in range(1, len(signal_pairs)):
            if( i == 1 ):
                #tuple1.append(())
                timediff = signal_pairs[i][0]-signal_pairs[0][0]
                signalTuple = (log(abs(max(amp,exp(0.1))),2), timediff)
                tuple1.append(signalTuple)
            else:
                timediff = signal_pairs[i][0]-signal_pairs[0][0]
                signalTuple = (log(abs(max(amp,exp(0.1))),2), timediff)
                tuple2.append(signalTuple)
        signal_count = signal_count + 1

    fig.suptitle(classification)
    timeTitle = '%f-%f'%(frontPeak/sampFreq, backPeak/sampFreq)
    fig.text(0,0,timeTitle)
    
    fname = ""
    
    if (classification == "Snap"):
        fname ='../Plots/Snaps/%f-%f.png'% (frontPeak/sampFreq, backPeak/sampFreq)
    elif (classification == "Ping"):
        fname ='../Plots/Pings/%f-%f.png'% (frontPeak/sampFreq, backPeak/sampFreq)
    else:
        fname ='../Plots/undefined/%f-%f.png'% (frontPeak/sampFreq, backPeak/sampFreq)
        
    
    #plt.show()
     
    #save image
    fig.savefig(fname)
    
    
    plt.close()

    
   

In [178]:
#Making the scatter plot
fig = plt.figure(figsize=(20, 5))
tuple1.sort(key = operator.itemgetter(0), reverse=True)
tuple2.sort(key = operator.itemgetter(0), reverse=True)
tuple1_amp = [x[0] for x in tuple1]
tuple2_amp = [x[0] for x in tuple2]
tuple1_timediff = [x[1] for x in tuple1]
tuple2_timediff = [x[1] for x in tuple2]

plt.scatter(tuple1_amp, tuple1_timediff, c ='b')
#plt.scatter(signal_id2, signal_timediff2, c ='r')
plt.title('Scatter plot')
plt.xlabel('log of amplitude of the reflected waves' )
plt.ylabel('arrival time diff in seconds')
plt.xlim(7,10)
plt.ylim(0,0.01)
plt.show()
